## 1. Install Required Packages

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.2 MB/s eta 0:00

## 2. Load Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("id_nergrit_corpus", "ner")

Generating train split:   0%|          | 0/12532 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2399 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2521 [00:00<?, ? examples/s]

Dataset id_nergrit_corpus downloaded and prepared to /root/.cache/huggingface/datasets/id_nergrit_corpus/ner/1.1.0/1acb30bb7756aced984d43315c7d928f030f2ad8719e601eaaa92e839162854e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 12532
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2399
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2521
    })
})

In [4]:
print(dataset['test'][0])
print(dataset['test'][3])

{'id': '0', 'tokens': ['Cianjur', 'Utara', ',', 'Destinasi', 'Wisata', 'Favorit', 'Saat', 'Ramadan'], 'ner_tags': [4, 23, 38, 38, 38, 38, 38, 16]}
{'id': '3', 'tokens': ['Ia', 'menjelaskan', ',', 'dinas', 'akan', 'fokus', 'menata', 'sistem', 'penjualan', 'tiket', 'yang', 'mempermudah', 'dan', 'menghindari', 'pungutan', '-', 'pungutan', 'lain', '.', 'Kondisi', 'sepinya', 'pengunjung', ',', 'dinilai', 'Asep', 'dapat', 'menjadi', 'kesempatan', 'untuk', 'megoptimalkan', 'penataan', 'tersebut', '.'], 'ner_tags': [38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38]}


In [5]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
print(label_list)
print(len(label_list))

['B-CRD', 'B-DAT', 'B-EVT', 'B-FAC', 'B-GPE', 'B-LAN', 'B-LAW', 'B-LOC', 'B-MON', 'B-NOR', 'B-ORD', 'B-ORG', 'B-PER', 'B-PRC', 'B-PRD', 'B-QTY', 'B-REG', 'B-TIM', 'B-WOA', 'I-CRD', 'I-DAT', 'I-EVT', 'I-FAC', 'I-GPE', 'I-LAN', 'I-LAW', 'I-LOC', 'I-MON', 'I-NOR', 'I-ORD', 'I-ORG', 'I-PER', 'I-PRC', 'I-PRD', 'I-QTY', 'I-REG', 'I-TIM', 'I-WOA', 'O']
39


The letter that prefixes each ner_tag indicates the token position of the entity:

* B- indicates the beginning of an entity.
* I- indicates a token is contained inside the same entity (for example, the State token is a part of an entity like Empire State Building).
* 0 indicates the token doesn’t correspond to any entity.

## 3. Pre-processing

In [6]:
from transformers import AutoTokenizer

# define the pre-trained model
model_name = 'indolem/indobert-base-uncased'
# define the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Realign the tokens

A single word corresponding to a single label may now be split into two subwords. We’ll need to realign the tokens and labels by:

* Mapping all tokens to their corresponding word with the word_ids method.
* Assigning the label -100 to the special tokens [CLS] and [SEP] so they’re ignored by the PyTorch loss function.
* Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Use Hugging Face Dataset map function to apply the pre-processing function over the entire dataset.

In [8]:
tokenized_txt = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/12532 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2399 [00:00<?, ? examples/s]

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Create data collator.

In [9]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

### Evaluation Metrics

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    report = classification_report(np.concatenate(true_labels), np.concatenate(true_predictions), digits=4)
    prec = precision_score(np.concatenate(true_labels), np.concatenate(true_predictions), average='macro')
    rec = recall_score(np.concatenate(true_labels), np.concatenate(true_predictions), average='macro')
    f1 = f1_score(np.concatenate(true_labels), np.concatenate(true_predictions), average='macro')
    acc = accuracy_score(np.concatenate(true_labels), np.concatenate(true_predictions))
    print("Test classification Report:\n{}".format(report))
    print("Overall Precision: ", prec)
    print("Overall Recall: ", rec)
    print("Overall F1 score: ", f1)
    print("Overall Accuracy: ", acc)
    return {
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "accuracy": acc,
    }

Create a map of the expected ids to their labels with id2label and label2id.

In [11]:
label2id = {t: i for i, t in enumerate(label_list)}
print(label2id)
id2label = {i: w for w, i in label2id.items()}
print(id2label)

{'B-CRD': 0, 'B-DAT': 1, 'B-EVT': 2, 'B-FAC': 3, 'B-GPE': 4, 'B-LAN': 5, 'B-LAW': 6, 'B-LOC': 7, 'B-MON': 8, 'B-NOR': 9, 'B-ORD': 10, 'B-ORG': 11, 'B-PER': 12, 'B-PRC': 13, 'B-PRD': 14, 'B-QTY': 15, 'B-REG': 16, 'B-TIM': 17, 'B-WOA': 18, 'I-CRD': 19, 'I-DAT': 20, 'I-EVT': 21, 'I-FAC': 22, 'I-GPE': 23, 'I-LAN': 24, 'I-LAW': 25, 'I-LOC': 26, 'I-MON': 27, 'I-NOR': 28, 'I-ORD': 29, 'I-ORG': 30, 'I-PER': 31, 'I-PRC': 32, 'I-PRD': 33, 'I-QTY': 34, 'I-REG': 35, 'I-TIM': 36, 'I-WOA': 37, 'O': 38}
{0: 'B-CRD', 1: 'B-DAT', 2: 'B-EVT', 3: 'B-FAC', 4: 'B-GPE', 5: 'B-LAN', 6: 'B-LAW', 7: 'B-LOC', 8: 'B-MON', 9: 'B-NOR', 10: 'B-ORD', 11: 'B-ORG', 12: 'B-PER', 13: 'B-PRC', 14: 'B-PRD', 15: 'B-QTY', 16: 'B-REG', 17: 'B-TIM', 18: 'B-WOA', 19: 'I-CRD', 20: 'I-DAT', 21: 'I-EVT', 22: 'I-FAC', 23: 'I-GPE', 24: 'I-LAN', 25: 'I-LAW', 26: 'I-LOC', 27: 'I-MON', 28: 'I-NOR', 29: 'I-ORD', 30: 'I-ORG', 31: 'I-PER', 32: 'I-PRC', 33: 'I-PRD', 34: 'I-QTY', 35: 'I-REG', 36: 'I-TIM', 37: 'I-WOA', 38: 'O'}


## 4. Fine-tuning with Trainer API

### Define the model

In [12]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    model_name, # we already defined the model name: indolem/indobert-base-uncased
    num_labels=39,
    id2label=id2label,
    label2id=label2id
)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

In [13]:
def train_predict_model(model_name, output_dir):

  training_args = TrainingArguments(
      output_dir=output_dir,
      logging_strategy="epoch",
      evaluation_strategy="epoch",
      save_strategy="epoch",
      save_total_limit = 1,
      learning_rate=2e-05,
      num_train_epochs=3,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      weight_decay=0.01,
      metric_for_best_model = "f1",
      load_best_model_at_end=True,
      #push_to_hub=True,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_txt['train'],
      eval_dataset=tokenized_txt['validation'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )
  print("Training Process")
  trainer.train()
  trainer.save_model(output_dir)

  # predict
  print("Prediction")
  pred_trainer = Trainer(
      model=model,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  pred_trainer.predict(tokenized_txt['test'])

In [14]:
train_predict_model(
    model_name=model_name,
    output_dir='ner_model'
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training Process


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.402800,0.186646,0.724052,0.698866,0.701783,0.943645
2,0.157800,0.176820,0.726824,0.735021,0.729091,0.948314
3,0.118300,0.181699,0.712938,0.739527,0.724108,0.947795


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Test classification Report:
              precision    recall  f1-score   support

       B-CRD     0.9089    0.9361    0.9223      1173
       B-DAT     0.9718    0.9718    0.9718       780
       B-EVT     0.6947    0.6701    0.6822       197
       B-FAC     0.5946    0.4583    0.5176        48
       B-GPE     0.9182    0.9277    0.9229      1258
       B-LAN     0.0000    0.0000    0.0000         1
       B-LAW     0.6750    0.6585    0.6667        82
       B-LOC     0.6254    0.6307    0.6280       352
       B-MON     0.9769    0.9826    0.9797       172
       B-NOR     0.7998    0.8444    0.8215       842
       B-ORD     0.7624    0.7476    0.7549       103
       B-ORG     0.7005    0.6955    0.6980       555
       B-PER     0.9457    0.9499    0.9478      1357
       B-PRC     0.9744    0.9845    0.9794       193
       B-PRD     0.7548    0.6374    0.6912       797
       B-QTY     0.8691    0.8020    0.8342       298
       B-REG     0.5238    0.4231    0.4681        52

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Test classification Report:
              precision    recall  f1-score   support

       B-CRD     0.9137    0.9386    0.9260      1173
       B-DAT     0.9743    0.9718    0.9730       780
       B-EVT     0.6995    0.7563    0.7268       197
       B-FAC     0.4306    0.6458    0.5167        48
       B-GPE     0.9193    0.9332    0.9262      1258
       B-LAN     0.0000    0.0000    0.0000         1
       B-LAW     0.6632    0.7683    0.7119        82
       B-LOC     0.6628    0.6534    0.6581       352
       B-MON     0.9826    0.9826    0.9826       172
       B-NOR     0.8015    0.8634    0.8313       842
       B-ORD     0.7453    0.7670    0.7560       103
       B-ORG     0.7930    0.6901    0.7380       555
       B-PER     0.9615    0.9558    0.9586      1357
       B-PRC     0.9845    0.9845    0.9845       193
       B-PRD     0.7357    0.7265    0.7311       797
       B-QTY     0.8597    0.8020    0.8299       298
       B-REG     0.6863    0.6731    0.6796        52

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Test classification Report:
              precision    recall  f1-score   support

       B-CRD     0.9137    0.9386    0.9260      1173
       B-DAT     0.9743    0.9718    0.9730       780
       B-EVT     0.7014    0.7513    0.7255       197
       B-FAC     0.4615    0.6250    0.5310        48
       B-GPE     0.9178    0.9404    0.9289      1258
       B-LAN     0.0000    0.0000    0.0000         1
       B-LAW     0.6214    0.7805    0.6919        82
       B-LOC     0.7073    0.6591    0.6824       352
       B-MON     0.9826    0.9826    0.9826       172
       B-NOR     0.8158    0.8729    0.8434       842
       B-ORD     0.7547    0.7767    0.7656       103
       B-ORG     0.7347    0.7333    0.7340       555
       B-PER     0.9599    0.9536    0.9567      1357
       B-PRC     0.9845    0.9845    0.9845       193
       B-PRD     0.7257    0.6939    0.7094       797
       B-QTY     0.8536    0.8020    0.8270       298
       B-REG     0.5775    0.7885    0.6667        52

Test classification Report:
              precision    recall  f1-score   support

       B-CRD     0.8487    0.9314    0.8882      1006
       B-DAT     0.9821    0.9772    0.9796       788
       B-EVT     0.7005    0.7582    0.7282       182
       B-FAC     0.4747    0.5054    0.4896        93
       B-GPE     0.8930    0.9041    0.8985      1376
       B-LAW     0.8571    0.6857    0.7619        35
       B-LOC     0.6452    0.6207    0.6327       290
       B-MON     0.9801    0.9647    0.9723       255
       B-NOR     0.8063    0.8439    0.8247       903
       B-ORD     0.9055    0.8156    0.8582       141
       B-ORG     0.7594    0.7541    0.7568       854
       B-PER     0.9824    0.9668    0.9746      1446
       B-PRC     0.9175    0.9844    0.9497       192
       B-PRD     0.7240    0.7765    0.7493       868
       B-QTY     0.8125    0.5756    0.6739       271
       B-REG     0.7000    0.6222    0.6588        45
       B-TIM     0.9912    0.9492    0.9697       118

## 5. Inference

In [15]:
from transformers import pipeline

text = ['Kompleks Keraton Jogja akan ditutup satu hari karena kunjungan Kaisar Jepang, Naruhito. Penutupan dilakukan pada Rabu besok (21/6/2023)',
        'Timnas asuhan Shin Tae Yong bermain imbang tanpa gol melawan Palestina yang menempati ranking 93 dunia, 14 Juni lalu']

for t in text:
  classifier = pipeline("ner", model="ner_model")
  print(classifier(t))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-LOC', 'score': 0.9281053, 'index': 1, 'word': 'kompleks', 'start': 0, 'end': 8}, {'entity': 'I-LOC', 'score': 0.89798474, 'index': 2, 'word': 'keraton', 'start': 9, 'end': 16}, {'entity': 'I-LOC', 'score': 0.9835032, 'index': 3, 'word': 'jogja', 'start': 17, 'end': 22}, {'entity': 'B-QTY', 'score': 0.9628866, 'index': 6, 'word': 'satu', 'start': 36, 'end': 40}, {'entity': 'I-QTY', 'score': 0.9414716, 'index': 7, 'word': 'hari', 'start': 41, 'end': 45}, {'entity': 'B-NOR', 'score': 0.58658594, 'index': 10, 'word': 'kaisar', 'start': 63, 'end': 69}, {'entity': 'I-NOR', 'score': 0.8002539, 'index': 11, 'word': 'jepang', 'start': 70, 'end': 76}, {'entity': 'B-PER', 'score': 0.9923962, 'index': 13, 'word': 'nar', 'start': 78, 'end': 81}, {'entity': 'I-PER', 'score': 0.9723412, 'index': 14, 'word': '##uhi', 'start': 81, 'end': 84}, {'entity': 'I-PER', 'score': 0.9805787, 'index': 15, 'word': '##to', 'start': 84, 'end': 86}, {'entity': 'B-DAT', 'score': 0.99714476, 'index': 20,

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-ORG', 'score': 0.9533131, 'index': 1, 'word': 'timnas', 'start': 0, 'end': 6}, {'entity': 'B-PER', 'score': 0.9423316, 'index': 3, 'word': 'shin', 'start': 14, 'end': 18}, {'entity': 'I-PER', 'score': 0.9370448, 'index': 4, 'word': 'tae', 'start': 19, 'end': 22}, {'entity': 'I-PER', 'score': 0.94891673, 'index': 5, 'word': 'yong', 'start': 23, 'end': 27}, {'entity': 'B-ORG', 'score': 0.8439367, 'index': 11, 'word': 'palestina', 'start': 61, 'end': 70}, {'entity': 'B-CRD', 'score': 0.897395, 'index': 15, 'word': '93', 'start': 94, 'end': 96}, {'entity': 'B-DAT', 'score': 0.99544406, 'index': 18, 'word': '14', 'start': 104, 'end': 106}, {'entity': 'I-DAT', 'score': 0.9963379, 'index': 19, 'word': 'juni', 'start': 107, 'end': 111}]
